In [1]:
import pandas as pd
import osmnx as ox # OSMnx is a Python package to get access to geospatial features from OpenStreetMap. (Boeing, G. 2024)
import geopandas as gpd
import numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import shap

In [2]:
## 6. POI Counts: 
# Select the number of transportation and shops in the area, 
# as they might act as crime attractors and influence crime opportunities.

# read lsoa 2011 boundaries
lsoa_gdf = gpd.read_file("data/gentri_data/london_gentri_labeled_25.shp")

# get poi data from open street map
# code from osmnx website
place = "London, UK"
tags1 = {"railway": "station", "highway": "bus_stop"}  # subway stations and bus stations
tags2 = {"shop": True } # shops
gdf1 = ox.features_from_place(place, tags1)
gdf2 = ox.features_from_place(place, tags2)

# set the same crs
tran_poi_gdf = gdf1.to_crs(lsoa_gdf.crs)
shop_poi_gdf = gdf2.to_crs(lsoa_gdf.crs)

# change all types of geospatial features into points
shop_poi_gdf["geometry"] = shop_poi_gdf.geometry.centroid

# join the transportation points with the lsoas
joined_tran = gpd.sjoin(tran_poi_gdf, lsoa_gdf, how='inner', predicate='within')
# counts the points within each lsoa
tran_poi_count = joined_tran.groupby('LSOA code').size().reset_index(name='tran_poi_count')

# same with the shop points
joined_shop = gpd.sjoin(shop_poi_gdf, lsoa_gdf, how='inner', predicate='within')
shop_poi_count = joined_shop.groupby('LSOA code').size().reset_index(name='shop_poi_count')

print(tran_poi_count.head(2))
print(shop_poi_count.head(2))

   LSOA code  tran_poi_count
0  E01000007               5
1  E01000008               1
   LSOA code  shop_poi_count
0  E01000005               1
1  E01000007              13


In [3]:
# import requests
# import json
# from shapely import wkt
# from tqdm import tqdm

# # 读取 LSOA 边界
# lsoa_gdf = gpd.read_file("data/gentri_data/london_gentri_labeled_25.shp")
# lsoa_gdf = lsoa_gdf.to_crs(4326)

# # 获取 London 边界
# london_geom_json = ox.geocode_to_gdf("London, UK").to_crs(4326).to_json()

# # 设置查询时间点和标签
# years = ["2015-01-01", "2019-01-01"]
# filters = {
#     "tran": "railway=station OR highway=bus_stop",
#     "shop": "shop=*"
# }

# # 正确的 endpoint
# def get_pois_ohsome(geometry, filter_str, time_str):
#     url = "https://api.ohsome.org/v1/elementsFull/geometry"  # ✅ FIXED
#     params = {
#         "bpolys": geometry,
#         "filter": filter_str,
#         "time": time_str,
#         "format": "geojson",
#         "types": "node,way"
#     }
#     response = requests.post(url, json=params)
#     response.raise_for_status()
#     data = response.json()
#     if not data["features"]:
#         print("⚠️ 结果为空")
#         return None
#     gdf = gpd.GeoDataFrame.from_features(data["features"])
#     gdf["timestamp"] = gdf["properties"].apply(lambda x: x["timestamp"])
#     return gdf.set_geometry("geometry", crs=4326)

# # 统计 LSOA 区内的 POI 数量
# def count_poi_per_lsoa(poi_gdf, lsoa_gdf, year_label):
#     poi_gdf = poi_gdf.copy()
#     poi_gdf["geometry"] = poi_gdf.geometry.centroid
#     joined = gpd.sjoin(poi_gdf.to_crs(lsoa_gdf.crs), lsoa_gdf, how='inner', predicate='within')
#     grouped = joined.groupby("LSOA code").size().reset_index(name=f"count_{year_label}")
#     return grouped

# # 主循环
# results = {}
# for key, filter_str in filters.items():
#     for year in years:
#         print(f"⏳ 查询 {key} 在 {year} 的 POI ...")
#         try:
#             poi_gdf = get_pois_ohsome(london_geom_json, filter_str, year)
#             if poi_gdf is None:
#                 continue
#             year_label = f"{key}_{year[:4]}"
#             count_df = count_poi_per_lsoa(poi_gdf, lsoa_gdf, year_label)
#             results[year_label] = count_df
#         except requests.exceptions.HTTPError as e:
#             print(f"❌ 请求失败：{e}")
#             print("响应内容:", e.response.text)
#             continue

# # 合并数据时确保 key 存在
# def safe_merge(df1, df2, on):
#     if df1 is None or df2 is None or df1.empty or df2.empty:
#         print("⚠️ 某些数据为空，跳过合并")
#         return pd.DataFrame()
#     return df1.merge(df2, on=on, how="outer").fillna(0)

# tran_df = safe_merge(results.get("tran_2015"), results.get("tran_2019"), on="LSOA code")
# shop_df = safe_merge(results.get("shop_2015"), results.get("shop_2019"), on="LSOA code")

# print("\n📊 交通 POI 统计（前几行）:")
# print(tran_df.head())

# print("\n📊 商铺 POI 统计（前几行）:")
# print(shop_df.head())

# # 保存文件
# if not tran_df.empty:
#     tran_df.to_csv("outputs/tran_poi_counts.csv", index=False)
# if not shop_df.empty:
#     shop_df.to_csv("outputs/shop_poi_counts.csv", index=False)

In [5]:
df = pd.read_csv("data/lsoa_model_dataframe.csv")

In [12]:
df_final = df.merge(tran_poi_count, on="LSOA code", how="left")
df_final = df_final.merge(shop_poi_count, on="LSOA code", how="left")
df_final["tran_poi_count"] = df_final["tran_poi_count"].fillna(0).astype(int)
df_final["shop_poi_count"] = df_final["shop_poi_count"].fillna(0).astype(int)

In [13]:
print(df_final.head(5))

   LSOA code  residential_topn_mean  commercial_topn_mean  green_topn_mean  \
0  E01002714               0.760136              0.689529         0.700867   
1  E01002771               0.781176              0.581556         0.683054   
2  E01000220               0.906424              0.508664         0.511596   
3  E01001220               0.669778              0.600520         0.555940   
4  E01003187               0.349043              0.578166         0.357311   

   cultural_topn_mean  infrustructure_topn_mean  gentrified       avg_den  \
0            0.580095                  0.756108           0  16340.515284   
1            0.564128                  0.699904           0  12654.580134   
2            0.580305                  0.633844           0  13904.333986   
3            0.555096                  0.644365           0   1436.788216   
4            0.474421                  0.488227           0  11407.115656   

   pop_growth_rate  senior_per  minority_per  tran_poi_count  shop_p

In [ ]:
features = ['pop_growth_rate', 'avg_den', 'senior_per', 'minority_per', 
            'residential_topn_mean', 'commercial_topn_mean', 
            'green_topn_mean', 'cultural_topn_mean', 'infrustructure_topn_mean',
            'tran_poi_count', 'shop_poi_count']
df_final['gentrified'] = df_final['gentrified'].astype(int)

X = df_final[features]
y = df_final['gentrified']

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

# 样本不均衡处理
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify=y, 
                                                    test_size=0.25, 
                                                    random_state=42)

scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])

model = XGBClassifier(scale_pos_weight = scale_pos_weight,
                      random_state=42, 
                      eval_metric="aucpr")

model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

from sklearn.metrics import roc_auc_score, average_precision_score

y_proba = model.predict_proba(X_test)[:, 1]  # 取预测为 1 的概率
print("AUROC:", roc_auc_score(y_test, y_proba))
print("AUPRC:", average_precision_score(y_test, y_proba))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96      1008
           1       0.11      0.03      0.05        60

    accuracy                           0.93      1068
   macro avg       0.52      0.51      0.51      1068
weighted avg       0.90      0.93      0.91      1068

AUROC: 0.6210648148148149
AUPRC: 0.09624990174113471


In [ ]:
from imblearn.under_sampling import RandomUnderSampler

X = df_final[features]
y = df_final['gentrified']

rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

X_resampled_train, X_resampled_test, y_resampled_train, y_resampled_test = train_test_split(X_resampled, y_resampled, 
                                                                                            stratify=y_resampled, 
                                                                                            test_size=0.25, 
                                                                                            random_state=42)

scale_pos_weight = len(y_resampled_train[y_resampled_train == 0]) / len(y_resampled_train[y_resampled_train == 1])

model_resampled = XGBClassifier(scale_pos_weight = scale_pos_weight,
                      random_state=42, 
                      eval_metric="aucpr")

model_resampled.fit(X_resampled_train, y_resampled_train)

# Evaluate
y_resampled_pred = model_resampled.predict(X_resampled_test)
print(classification_report(y_resampled_test, y_resampled_pred))
y_resampled_proba = model_resampled.predict_proba(X_resampled_test)[:, 1]  # 取预测为 1 的概率
print("AUROC:", roc_auc_score(y_resampled_test, y_resampled_proba))
print("AUPRC:", average_precision_score(y_resampled_test, y_resampled_proba))

              precision    recall  f1-score   support

           0       0.54      0.53      0.54        60
           1       0.54      0.55      0.55        60

    accuracy                           0.54       120
   macro avg       0.54      0.54      0.54       120
weighted avg       0.54      0.54      0.54       120

AUROC: 0.6108333333333333
AUPRC: 0.6059215597526756
